In [1]:
import pandas as pd
import numpy as np

# First we load in population data for all federal states and all calculations

In [2]:
data=pd.read_csv("RawData/12411-0013.csv",encoding="latin",sep=";",skiprows=4)
data=data.drop("Unnamed: 0",axis=1)
data=data.rename(columns={"Unnamed: 1":"Alter"}).dropna()

#and we clean the column by combining the first and the second row

#so we get all the federal states and remove the numbering for duplicate column names
bl=data.columns.tolist()
bl_list=[]

for test_str in bl:
    test_str=test_str.split(".")[0]
    bl_list.append(test_str)

#so bl_list is a cleaned version of all the federal states from row 1 
#then we do the same for the row below

data=pd.read_csv("RawData/12411-0013.csv",encoding="latin",sep=";",skiprows=5)
data=data.drop("Unnamed: 0",axis=1)
data=data.rename(columns={"Unnamed: 1":"Alter"}).dropna()
geschlecht=data.columns.tolist()
geschlecht_list=[]

index=0
for test_str in geschlecht:
    test_str=test_str.split(".")[0]
    #but we append directly to the first row name the second row name
    geschlecht_list.append(bl_list[index]+"_"+test_str)
    index+=1
geschlecht_list
#and then set this to be the columnnames of the dataframe
data.columns=geschlecht_list

#then we remove the summed row at the end
data=data.iloc[:91]
#and we replace "Jährige" etc. to have an integer column for age
data["Alter_Alter"]=data["Alter_Alter"].str.replace("-Jährige","").replace("90 Jahre und mehr","90").replace("unter 1 Jahr","0")
data["Alter_Alter"]=data["Alter_Alter"].astype(int)
raw_data=data
pop_base=raw_data[raw_data.columns[raw_data.columns.str.contains("Insgesamt")]]
pop_base=pop_base.reset_index().rename(columns={"index":"Alter"})

#then we apply a very simple model that estimates that 80 percent of everyone older than 65 would have to get covid
#and 95% of everyone younger than 65 would have to get covid,
#to reach herd immunity
upper=data[data["Alter_Alter"]>=65]
upper=upper.iloc[:,1:]*0.8
lower=data[data["Alter_Alter"]<65]
lower=lower.iloc[:,1:]*0.95
pop_data=pd.DataFrame()
pop_data=pd.concat([lower,upper])
pop_data["Alter_Alter"]=data["Alter_Alter"]
cols = list(pop_data)
# move the column to head of list using index, pop and insert
cols.insert(0, cols.pop(cols.index('Alter_Alter')))
pop_data=pop_data[cols]
pop_data

,Alter_Alter,Baden-Württemberg_männlich,Baden-Württemberg_weiblich,Baden-Württemberg_Insgesamt,Bayern_männlich,Bayern_weiblich,Bayern_Insgesamt,Berlin_männlich,Berlin_weiblich,Berlin_Insgesamt,...,Sachsen_Insgesamt,Sachsen-Anhalt_männlich,Sachsen-Anhalt_weiblich,Sachsen-Anhalt_Insgesamt,Schleswig-Holstein_männlich,Schleswig-Holstein_weiblich,Schleswig-Holstein_Insgesamt,Thüringen_männlich,Thüringen_weiblich,Thüringen_Insgesamt
0,0,52735.45,50152.40,102887.85,62372.25,59261.95,121634.20,18932.55,17793.50,36726.05,...,32657.20,8097.80,7670.30,15768.10,12071.65,11457.00,23528.65,7933.45,7823.25,15756.70
1,1,53554.35,51118.55,104672.90,62732.30,59821.50,122553.80,18979.10,18092.75,37071.85,...,34208.55,8498.70,8179.50,16678.20,12516.25,11981.40,24497.65,8574.70,8119.65,16694.35
2,2,53667.40,50934.25,104601.65,62644.90,60238.55,122883.45,19172.90,18021.50,37194.40,...,35653.50,8978.45,8483.50,17461.95,12767.05,12077.35,24844.40,8943.30,8494.90,17438.20
3,3,53645.55,50778.45,104424.00,62605.00,59439.60,122044.60,18984.80,18266.60,37251.40,...,36519.90,9178.90,8609.85,17788.75,12895.30,12407.00,25302.30,9163.70,8801.75,17965.45
4,4,51484.30,49740.10,101224.40,60378.20,57498.75,117876.95,18245.70,17082.90,35328.60,...,36161.75,8952.80,8618.40,17571.20,12637.85,11964.30,24602.15,9292.90,8673.50,17966.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,86,13272.80,20798.40,34071.20,14806.40,24276.80,39083.20,3393.60,5784.80,9178.40,...,15975.20,2869.60,5529.60,8399.20,3316.00,5472.00,8788.00,2784.80,4820.00,7604.80
87,87,11310.40,18943.20,30253.60,12675.20,22424.00,35099.20,2828.00,5004.80,7832.80,...,13882.40,2521.60,4902.40,7424.00,2776.00,4832.80,7608.80,2336.00,4482.40,6818.40
88,88,9940.00,17228.80,27168.80,10896.80,20076.00,30972.80,2546.40,4580.80,7127.20,...,12899.20,2109.60,4519.20,6628.80,2431.20,4621.60,7052.80,1979.20,3972.00,5951.20
89,89,8596.00,15994.40,24590.40,9429.60,18464.00,27893.60,2191.20,4408.80,6600.00,...,12139.20,1789.60,4203.20,5992.80,2103.20,4132.80,6236.00,1649.60,3847.20,5496.80


# A) [Nature Paper](https://static-content.springer.com/esm/art%3A10.1038%2Fs41586-020-2918-0/MediaObjects/41586_2020_2918_MOESM1_ESM.pdf)

## First we do the calculation of how many would die, if all people get Covid. This calculation accounts for gender differences in fatality.

In [3]:
ifr_nature_w=pd.read_excel("RawData/ifr.xlsx",sheet_name="nature_w",engine='openpyxl')
ifr_nature_m=pd.read_excel("RawData/ifr.xlsx",sheet_name="nature_m",engine='openpyxl')
ifr_nature=pd.merge(ifr_nature_w,ifr_nature_m,on=["Junges Alter","Altes Alter"],suffixes=["_weiblich","_männlich"])
ifr_nature

,Junges Alter,Altes Alter,Mean_weiblich,Min_weiblich,Max_weiblich,Mean_männlich,Min_männlich,Max_männlich
0,0,4,0.003,0.002,0.003,0.003,0.002,0.004
1,5,9,0.001,0.000,0.001,0.001,0.000,0.001
2,10,14,0.001,0.000,0.001,0.001,0.001,0.002
3,15,19,0.002,0.002,0.003,0.003,0.002,0.003
4,20,24,0.005,0.004,0.006,0.008,0.007,0.009
5,25,29,0.009,0.008,0.011,0.017,0.014,0.020
6,30,34,0.015,0.013,0.018,0.033,0.028,0.038
7,35,39,0.025,0.021,0.029,0.056,0.048,0.065
8,40,44,0.044,0.038,0.051,0.106,0.091,0.123
9,45,49,0.073,0.063,0.085,0.168,0.144,0.195


In [4]:
#now we filter the population data by stripping away the sum of both genders
male_female_pop=pop_data[pop_data.columns[~pop_data.columns.str.contains("Insgesamt")]]
male_female_pop

,Alter_Alter,Baden-Württemberg_männlich,Baden-Württemberg_weiblich,Bayern_männlich,Bayern_weiblich,Berlin_männlich,Berlin_weiblich,Brandenburg_männlich,Brandenburg_weiblich,Bremen_männlich,...,Saarland_männlich,Saarland_weiblich,Sachsen_männlich,Sachsen_weiblich,Sachsen-Anhalt_männlich,Sachsen-Anhalt_weiblich,Schleswig-Holstein_männlich,Schleswig-Holstein_weiblich,Thüringen_männlich,Thüringen_weiblich
0,0,52735.45,50152.40,62372.25,59261.95,18932.55,17793.50,9589.30,9058.25,3507.40,...,3889.30,3701.20,16710.50,15946.70,8097.80,7670.30,12071.65,11457.00,7933.45,7823.25
1,1,53554.35,51118.55,62732.30,59821.50,18979.10,18092.75,10323.65,9690.95,3381.05,...,4009.95,3734.45,17565.50,16643.05,8498.70,8179.50,12516.25,11981.40,8574.70,8119.65
2,2,53667.40,50934.25,62644.90,60238.55,19172.90,18021.50,10779.65,10369.25,3347.80,...,4074.55,3912.10,18194.40,17459.10,8978.45,8483.50,12767.05,12077.35,8943.30,8494.90
3,3,53645.55,50778.45,62605.00,59439.60,18984.80,18266.60,11307.85,10735.00,3275.60,...,3990.95,3980.50,18676.05,17843.85,9178.90,8609.85,12895.30,12407.00,9163.70,8801.75
4,4,51484.30,49740.10,60378.20,57498.75,18245.70,17082.90,11090.30,10333.15,3260.40,...,4055.55,3633.75,18698.85,17462.90,8952.80,8618.40,12637.85,11964.30,9292.90,8673.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,86,13272.80,20798.40,14806.40,24276.80,3393.60,5784.80,3304.80,5664.00,738.40,...,1362.40,2228.00,5820.00,10155.20,2869.60,5529.60,3316.00,5472.00,2784.80,4820.00
87,87,11310.40,18943.20,12675.20,22424.00,2828.00,5004.80,2879.20,5151.20,591.20,...,1167.20,2117.60,4849.60,9032.80,2521.60,4902.40,2776.00,4832.80,2336.00,4482.40
88,88,9940.00,17228.80,10896.80,20076.00,2546.40,4580.80,2456.00,4632.00,540.80,...,1009.60,1935.20,4327.20,8572.00,2109.60,4519.20,2431.20,4621.60,1979.20,3972.00
89,89,8596.00,15994.40,9429.60,18464.00,2191.20,4408.80,2113.60,4300.00,452.80,...,848.00,1649.60,3838.40,8300.80,1789.60,4203.20,2103.20,4132.80,1649.60,3847.20


In [5]:
#we prepare lists of series for the mean, max, min number of deaths for male and female

mean_series_male=[]
mean_series_female=[]
max_series_male=[]
max_series_female=[]
min_series_male=[]
min_series_female=[]

#and load in the ifr from the nature paper for the age groups
ifr=ifr_nature
for index, row in male_female_pop.iterrows():
    #we retrieve the age of that row
    altersgruppe=row["Alter_Alter"]
    #and then get the row from the nature paper that states the mortality in this age group
    mortality_numbers=ifr[(ifr["Junges Alter"]<=altersgruppe)].iloc[-1]
    
    #so here we multiply the number of men and women with the ifr to estimate the number of potential deaths
    mean_series_male.append(((row[row.index.str.contains("männlich")]*mortality_numbers["Mean_männlich"]*0.01)))
    mean_series_female.append(((row[row.index.str.contains("weiblich")]*mortality_numbers["Mean_weiblich"]*0.01)))
    max_series_male.append(((row[row.index.str.contains("männlich")]*mortality_numbers["Max_männlich"]*0.01)))
    max_series_female.append(((row[row.index.str.contains("weiblich")]*mortality_numbers["Max_weiblich"]*0.01)))
    min_series_male.append(((row[row.index.str.contains("männlich")]*mortality_numbers["Min_männlich"]*0.01)))
    min_series_female.append(((row[row.index.str.contains("weiblich")]*mortality_numbers["Min_weiblich"]*0.01)))

    
#we build dataframes from these lists of series     
mean_male = pd.concat(mean_series_male, axis=1).T
mean_female = pd.concat(mean_series_female, axis=1).T
max_male = pd.concat(max_series_male, axis=1).T
max_female = pd.concat(max_series_female, axis=1).T
min_male = pd.concat(min_series_male, axis=1).T
min_female = pd.concat(min_series_female, axis=1).T

#and then we calculate a df for each scenario in which we just add deaths of male and female potentials deaths
mean_df=pd.DataFrame()
min_df=pd.DataFrame()
max_df=pd.DataFrame()

columnlist_gesamt = [item[0] for item in mean_male .columns.str.split("_")]

for i in columnlist_gesamt:
    mean_df[(i)+"_Insgesamt"]=mean_male[i+"_männlich"]+mean_female[i+"_weiblich"]
    min_df[(i)+"_Insgesamt"]=min_male[i+"_männlich"]+min_female[i+"_weiblich"]
    max_df[(i)+"_Insgesamt"]=max_male[i+"_männlich"]+max_female[i+"_weiblich"]
    
    
mean_df=mean_df.reset_index().rename(columns={"index":'Alter'})
min_df=min_df.reset_index().rename(columns={"index":'Alter'})
max_df=max_df.reset_index().rename(columns={"index":'Alter'})
mean_df.sum()

Alter                                 4095.000000
Baden-Württemberg_Insgesamt          77198.079760
Bayern_Insgesamt                     90673.086086
Berlin_Insgesamt                     23414.161388
Brandenburg_Insgesamt                21043.244947
Bremen_Insgesamt                      4817.942036
Hamburg_Insgesamt                    11543.730180
Hessen_Insgesamt                     43931.171604
Mecklenburg-Vorpommern_Insgesamt     13345.851124
Niedersachsen_Insgesamt              59467.398454
Nordrhein-Westfalen_Insgesamt       128736.486102
Rheinland-Pfalz_Insgesamt            30194.435059
Saarland_Insgesamt                    7925.063228
Sachsen_Insgesamt                    35789.258902
Sachsen-Anhalt_Insgesamt             19226.556333
Schleswig-Holstein_Insgesamt         22562.848880
Thüringen_Insgesamt                  18121.916659
dtype: float64

## So we know now how many people would die in each federal state and in each age group. Now, we simply have to translate this into how many people have been saved thanks to a vaccine.

In [6]:
#Now we want to calculate the number of saved lives
#we build a dictionary that connects the abreviations to the spelled out name of all federal staes
BL_dict={"BE":"Berlin","BB":"Brandenburg","BW":"Baden-Württemberg","BY":"Bayern",
         "HB":"Bremen","HE":"Hessen","HH":"Hamburg","MV":"Mecklenburg-Vorpommern",
         "NI":"Niedersachsen","NW":"Nordrhein-Westfalen","RP":"Rheinland-Pfalz","SH":"Schleswig-Holstein",
        "SL":"Saarland","SN":"Sachsen","ST":"Sachsen-Anhalt","TH":"Thüringen"}

#we split the population into two groups, people over 80 (because these are the ones 
#prioritized for vaccination by age 
pop_alter=pop_data[pop_data["Alter_Alter"]>=80].iloc[:,1:].sum()
#and the rest which is between 18 and 79 old and sum over these groups for each column
pop_rest=pop_data[(pop_data["Alter_Alter"]>=18)&(pop_data["Alter_Alter"]<80)].iloc[:,1:].sum()

#and we do the same for the potential covid deaths for each scenario split by age group
mean_df_alter=mean_df[mean_df["Alter"]>=80].iloc[:,1:].sum()
mean_df_rest=mean_df[(mean_df["Alter"]>=18)&(mean_df["Alter"]<80)].iloc[:,1:].sum()
max_df_alter=max_df[max_df["Alter"]>=80].iloc[:,1:].sum()
max_df_rest=max_df[(max_df["Alter"]>=18)&(max_df["Alter"]<80)].iloc[:,1:].sum()
min_df_alter=min_df[min_df["Alter"]>=80].iloc[:,1:].sum()
min_df_rest=min_df[(min_df["Alter"]>=18)&(min_df["Alter"]<80)].iloc[:,1:].sum()
min_df_rest
#these numbers we could actually also store as a factor, to update constantly a dashboard
coefficients_nature=pd.concat([mean_df_alter,mean_df_rest], axis=1).reset_index().rename(columns={"index":"Bundesland",0:"Alter",1:"Rest"})
coefficients_nature

,Bundesland,Alter,Rest
0,Baden-Württemberg_Insgesamt,44594.165504,32571.093485
1,Bayern_Insgesamt,51322.038504,39313.180753
2,Berlin_Insgesamt,12955.328520,10448.162991
3,Brandenburg_Insgesamt,12105.868048,8930.610315
4,Bremen_Insgesamt,2777.511536,2038.474497
5,Hamburg_Insgesamt,6598.362824,4939.848778
6,Hessen_Insgesamt,24994.721512,18918.090744
7,Mecklenburg-Vorpommern_Insgesamt,7694.834576,5646.836709
8,Niedersachsen_Insgesamt,34159.445920,25284.796150
9,Nordrhein-Westfalen_Insgesamt,74668.977480,54015.044454


In [7]:

#again we build lists of series for mean, min, max
series_of_lives_mean=[]
series_of_lives_min=[]
series_of_lives_max=[]
#and also have one list for the column names
column_names=[]

#we go through the BL dictionary
for BL in (BL_dict.keys()):
    #retrieve the latest data for vaccinated people
    geimpfte=pd.read_csv("https://raw.githubusercontent.com/ard-data/2020-rki-impf-archive/master/data/9_csv_v2/region_"+BL+".csv")
    #and only filter for the sum of vaccinated people with two jabs 
    #and the subgroup of these people that were vaccinated because of age (over 80)
    geimpfte_erst_indikation=geimpfte[["personen_erst_kumulativ","indikation_alter_erst","personen_voll_kumulativ","indikation_alter_voll","impf_inzidenz_voll","impf_inzidenz_erst"]]
    #and we get the under 80 year olds by a simple substraction
    geimpfte_erst_indikation["indikation_rest_voll"]=geimpfte_erst_indikation["personen_voll_kumulativ"]-geimpfte_erst_indikation["indikation_alter_voll"]
    
    #and then we calculate which share of the over 80 population and which share of the other population has been vaccinated
    share_alter_geimpfte=geimpfte_erst_indikation["indikation_alter_voll"]/pop_alter[BL_dict[BL]+"_Insgesamt"]   
    share_rest_geimpfte=geimpfte_erst_indikation["indikation_rest_voll"]/pop_rest[BL_dict[BL]+"_Insgesamt"]
    
    #and calculate the savedlives by multiplying this share with the potential deaths in each age group for that federal state
    savedlives_mean_alter=(share_alter_geimpfte*mean_df_alter[BL_dict[BL]+"_Insgesamt"])*0.99
    savedlives_mean_rest=(share_rest_geimpfte*mean_df_rest[BL_dict[BL]+"_Insgesamt"] )*0.99   
    series_of_lives_mean.extend([savedlives_mean_alter,savedlives_mean_rest])
    
    savedlives_min_alter=(share_alter_geimpfte*min_df_alter[BL_dict[BL]+"_Insgesamt"])*0.99
    savedlives_min_rest=(share_rest_geimpfte*min_df_rest[BL_dict[BL]+"_Insgesamt"])*0.99
    series_of_lives_min.extend([savedlives_min_alter,savedlives_min_rest])
    
    savedlives_max_alter=(share_alter_geimpfte*max_df_alter[BL_dict[BL]+"_Insgesamt"])*0.99
    savedlives_max_rest=(share_rest_geimpfte*max_df_rest[BL_dict[BL]+"_Insgesamt"])*0.99    
    series_of_lives_max.extend([savedlives_max_alter,savedlives_max_rest])
    
    column_names.extend([BL_dict[BL]+"_alter",BL_dict[BL]+"_rest"])
    
saved_lives_mean_df=pd.concat(series_of_lives_mean,axis=1)   
saved_lives_mean_df.columns=column_names
saved_lives_max_df=pd.concat(series_of_lives_max,axis=1)   
saved_lives_max_df.columns=column_names
saved_lives_min_df=pd.concat(series_of_lives_min,axis=1)   
saved_lives_min_df.columns=column_names
saved_lives_min_df
print(saved_lives_min_df.sum(axis=1).iloc[-1],saved_lives_mean_df.sum(axis=1).iloc[-1],saved_lives_max_df.sum(axis=1).iloc[-1])
nature_calc=saved_lives_mean_df
nature_calc

<ipython-input-7-e3b95944cc80>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  geimpfte_erst_indikation["indikation_rest_voll"]=geimpfte_erst_indikation["personen_erst_kumulativ"]-geimpfte_erst_indikation["indikation_alter_voll"]


45625.2265054497 53294.082378309744 61592.80319848189


,Berlin_alter,Berlin_rest,Brandenburg_alter,Brandenburg_rest,Baden-Württemberg_alter,Baden-Württemberg_rest,Bayern_alter,Bayern_rest,Bremen_alter,Bremen_rest,...,Schleswig-Holstein_alter,Schleswig-Holstein_rest,Saarland_alter,Saarland_rest,Sachsen_alter,Sachsen_rest,Sachsen-Anhalt_alter,Sachsen-Anhalt_rest,Thüringen_alter,Thüringen_rest
0,0.000000,6.381341,0.000000,0.500763,0.000000,8.565378,0.000000,14.113385,0.000000,1.752285,...,0.000000,3.795236,0.000000,0.993840,0.000000,0.928051,0.000000,7.890971,0.000000,0.508577
1,0.000000,15.066400,0.000000,0.535816,0.000000,19.403623,0.000000,21.734363,0.000000,2.469888,...,0.000000,11.704714,0.000000,3.984604,0.000000,4.369998,0.000000,20.907925,0.000000,1.320246
2,0.000000,35.650737,0.000000,15.203153,0.000000,51.988228,0.000000,117.463008,0.000000,7.055035,...,0.000000,26.181968,0.000000,9.795100,0.000000,13.528127,0.000000,52.991228,0.000000,4.161088
3,0.000000,43.617601,0.000000,16.119549,0.000000,70.224592,0.000000,158.062414,0.000000,7.263640,...,0.000000,34.105521,0.000000,12.554739,0.000000,16.776305,0.000000,58.479231,0.000000,4.161088
4,0.000000,51.556993,0.000000,16.119549,0.000000,82.386278,0.000000,162.435107,0.000000,7.263640,...,0.000000,37.361261,0.000000,15.328245,0.000000,20.396723,0.000000,61.758391,0.000000,4.161088
5,0.000000,57.361423,0.000000,16.119549,0.000000,98.900524,0.000000,240.843726,0.000000,7.664162,...,0.000000,41.907100,0.000000,19.178796,0.000000,22.145742,0.000000,67.272627,0.000000,4.161088
6,0.000000,69.692402,0.000000,16.570235,0.000000,112.837759,0.000000,275.929376,0.000000,7.939522,...,0.000000,44.834451,0.000000,19.178796,0.000000,24.812614,0.000000,70.126808,0.000000,4.161088
7,0.000000,76.093366,0.000000,16.695426,0.000000,132.270152,0.000000,324.312174,0.000000,11.068603,...,0.000000,60.883280,0.000000,22.354461,0.000000,39.467660,0.000000,81.994744,0.000000,13.947350
8,0.000000,86.163796,0.000000,27.061212,0.000000,155.524907,0.000000,344.605631,0.000000,13.329885,...,0.000000,79.826622,0.000000,26.038602,0.000000,54.902880,0.000000,89.838496,0.000000,19.063947
9,0.000000,94.813535,0.000000,40.972397,0.000000,176.317732,0.000000,351.268781,0.000000,15.958313,...,0.000000,94.191285,0.000000,30.097166,0.000000,66.319946,0.000000,92.467070,0.000000,27.447768


# B) [Medrxiv paper](https://www.medrxiv.org/content/10.1101/2020.07.23.20160895v7.full.pdf)
## Then we do the same calculation of how many would have died, if all had Covid based on another paper 

In [8]:
ifr_medrxiv=pd.read_excel("RawData/ifr.xlsx",sheet_name="medrxiv",engine='openpyxl')
ifr_medrxiv

,Junges Alter,Altes Alter,Mean,Min,Max
0,0,34,0.004,0.003,0.005
1,35,44,0.068,0.058,0.078
2,45,54,0.230,0.200,0.260
3,55,64,0.750,0.660,0.870
4,65,74,2.500,2.100,3.000
5,75,84,8.500,6.900,10.400
6,85,100,28.300,21.800,36.600


In [9]:
#we build a list of series
mean_series=[]
min_series=[]
max_series=[]
ifr=ifr_medrxiv
#for each row in the population data
for index, row in pop_data.iterrows():
    #we retrieve the age of that row
    altersgruppe=row["Alter_Alter"]
    #and then get the row from the nature paper that states the mortality in this age group
    mortality_numbers=ifr[(ifr["Junges Alter"]<=altersgruppe)].iloc[-1]
    #and we multiply the each row with the mean, min and max value of mortality in percent with
    #all the population data in the row. this way we get the number of people that would die
    #if all people in this age group would get covid, and we append this row/series to a list
    mean_series.append(row[pop_data.columns[1:]]*mortality_numbers["Mean"]*0.01)
    min_series.append(row[pop_data.columns[1:]]*mortality_numbers["Min"]*0.01)
    max_series.append(row[pop_data.columns[1:]]*mortality_numbers["Max"]*0.01)

#from these lists we build dataframes 
mean_df = pd.concat(mean_series, axis=1).T
min_df = pd.concat(min_series, axis=1).T
max_df = pd.concat(max_series, axis=1).T
mean_df=mean_df.reset_index().rename(columns={"index":'Alter'})
min_df=min_df.reset_index().rename(columns={"index":'Alter'})
max_df=max_df.reset_index().rename(columns={"index":'Alter'})

max_df

,Alter,Baden-Württemberg_männlich,Baden-Württemberg_weiblich,Baden-Württemberg_Insgesamt,Bayern_männlich,Bayern_weiblich,Bayern_Insgesamt,Berlin_männlich,Berlin_weiblich,Berlin_Insgesamt,...,Sachsen_Insgesamt,Sachsen-Anhalt_männlich,Sachsen-Anhalt_weiblich,Sachsen-Anhalt_Insgesamt,Schleswig-Holstein_männlich,Schleswig-Holstein_weiblich,Schleswig-Holstein_Insgesamt,Thüringen_männlich,Thüringen_weiblich,Thüringen_Insgesamt
0,0,2.636773,2.507620,5.144393,3.118612,2.963097,6.081710,0.946628,0.889675,1.836302,...,1.632860,0.404890,0.383515,0.788405,0.603583,0.572850,1.176432,0.396673,0.391163,0.787835
1,1,2.677717,2.555928,5.233645,3.136615,2.991075,6.127690,0.948955,0.904638,1.853592,...,1.710427,0.424935,0.408975,0.833910,0.625813,0.599070,1.224882,0.428735,0.405983,0.834718
2,2,2.683370,2.546713,5.230082,3.132245,3.011928,6.144172,0.958645,0.901075,1.859720,...,1.782675,0.448923,0.424175,0.873098,0.638352,0.603867,1.242220,0.447165,0.424745,0.871910
3,3,2.682277,2.538922,5.221200,3.130250,2.971980,6.102230,0.949240,0.913330,1.862570,...,1.825995,0.458945,0.430493,0.889438,0.644765,0.620350,1.265115,0.458185,0.440088,0.898273
4,4,2.574215,2.487005,5.061220,3.018910,2.874938,5.893848,0.912285,0.854145,1.766430,...,1.808088,0.447640,0.430920,0.878560,0.631892,0.598215,1.230107,0.464645,0.433675,0.898320
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,86,4857.844800,7612.214400,12470.059200,5419.142400,8885.308800,14304.451200,1242.057600,2117.236800,3359.294400,...,5846.923200,1050.273600,2023.833600,3074.107200,1213.656000,2002.752000,3216.408000,1019.236800,1764.120000,2783.356800
87,87,4139.606400,6933.211200,11072.817600,4639.123200,8207.184000,12846.307200,1035.048000,1831.756800,2866.804800,...,5080.958400,922.905600,1794.278400,2717.184000,1016.016000,1768.804800,2784.820800,854.976000,1640.558400,2495.534400
88,88,3638.040000,6305.740800,9943.780800,3988.228800,7347.816000,11336.044800,931.982400,1676.572800,2608.555200,...,4721.107200,772.113600,1654.027200,2426.140800,889.819200,1691.505600,2581.324800,724.387200,1453.752000,2178.139200
89,89,3146.136000,5853.950400,9000.086400,3451.233600,6757.824000,10209.057600,801.979200,1613.620800,2415.600000,...,4442.947200,654.993600,1538.371200,2193.364800,769.771200,1512.604800,2282.376000,603.753600,1408.075200,2011.828800


In [10]:
#Now we want to calculate the number of saved lives
#we build a dictionary that connects the abreviations to the spelled out name of all federal staes
BL_dict={"BE":"Berlin","BB":"Brandenburg","BW":"Baden-Württemberg","BY":"Bayern",
         "HB":"Bremen","HE":"Hessen","HH":"Hamburg","MV":"Mecklenburg-Vorpommern",
         "NI":"Niedersachsen","NW":"Nordrhein-Westfalen","RP":"Rheinland-Pfalz","SH":"Schleswig-Holstein",
        "SL":"Saarland","SN":"Sachsen","ST":"Sachsen-Anhalt","TH":"Thüringen"}

#we split the population into two groups, people over 80 (because these are the ones 
#prioritized for vaccination by age 
pop_alter=pop_data[pop_data["Alter_Alter"]>=80].iloc[:,1:].sum()
#and the rest which is between 18 and 79 old and sum over these groups for each column
pop_rest=pop_data[(pop_data["Alter_Alter"]>=18)&(pop_data["Alter_Alter"]<80)].iloc[:,1:].sum()

#and we do the same for the potential covid deaths for each scenario split by age group
mean_df_alter=mean_df[mean_df["Alter"]>=80].iloc[:,1:].sum()
mean_df_rest=mean_df[(mean_df["Alter"]>=18)&(mean_df["Alter"]<80)].iloc[:,1:].sum()
max_df_alter=max_df[max_df["Alter"]>=80].iloc[:,1:].sum()
max_df_rest=max_df[(max_df["Alter"]>=18)&(max_df["Alter"]<80)].iloc[:,1:].sum()
min_df_alter=min_df[min_df["Alter"]>=80].iloc[:,1:].sum()
min_df_rest=min_df[(min_df["Alter"]>=18)&(min_df["Alter"]<80)].iloc[:,1:].sum()
min_df_rest
#these numbers we could actually also store as a factor, to update constantly a dashboard
coefficients_medrxiv=pd.concat([mean_df_alter,mean_df_rest], axis=1).reset_index().rename(columns={"index":"Bundesland",0:"Alter",1:"Rest"})
coefficients_medrxiv=coefficients_medrxiv[coefficients_medrxiv["Bundesland"].str.contains("Insgesamt")]
coefficients_medrxiv

,Bundesland,Alter,Rest
2,Baden-Württemberg_Insgesamt,97465.6032,69547.077891
5,Bayern_Insgesamt,112000.9400,83750.194949
8,Berlin_Insgesamt,27986.7864,22721.072515
11,Brandenburg_Insgesamt,25967.9760,19621.761867
14,Bremen_Insgesamt,6140.1024,4386.193164
17,Hamburg_Insgesamt,14537.5896,10582.036565
20,Hessen_Insgesamt,54775.6840,40304.666112
23,Mecklenburg-Vorpommern_Insgesamt,16734.7296,12360.386343
26,Niedersachsen_Insgesamt,74332.5680,54155.566319
29,Nordrhein-Westfalen_Insgesamt,165074.5240,115841.081806


In [11]:

#again we build lists of series for mean, min, max
series_of_lives_mean=[]
series_of_lives_min=[]
series_of_lives_max=[]
#and also have one list for the column names
column_names=[]

#we go through the BL dictionary
for BL in (BL_dict.keys()):
    #retrieve the latest data for vaccinated people
    geimpfte=pd.read_csv("https://raw.githubusercontent.com/ard-data/2020-rki-impf-archive/master/data/9_csv_v2/region_"+BL+".csv")
    #and only filter for the sum of vaccinated people with two jabs 
    #and the subgroup of these people that were vaccinated because of age (over 80)
    geimpfte_erst_indikation=geimpfte[["personen_erst_kumulativ","indikation_alter_erst","personen_voll_kumulativ","indikation_alter_voll","impf_inzidenz_voll","impf_inzidenz_erst"]]
    #and we get the under 80 year olds by a simple substraction
    geimpfte_erst_indikation["indikation_rest_voll"]=geimpfte_erst_indikation["personen_voll_kumulativ"]-geimpfte_erst_indikation["indikation_alter_voll"]
    
    #and then we calculate which share of the over 80 population and which share of the other population has been vaccinated
    share_alter_geimpfte=geimpfte_erst_indikation["indikation_alter_voll"]/pop_alter[BL_dict[BL]+"_Insgesamt"]   
    share_rest_geimpfte=geimpfte_erst_indikation["indikation_rest_voll"]/pop_rest[BL_dict[BL]+"_Insgesamt"]
    
    #and calculate the savedlives by multiplying this share with the potential deaths in each age group for that federal state
    savedlives_mean_alter=(share_alter_geimpfte*mean_df_alter[BL_dict[BL]+"_Insgesamt"])*0.99
    savedlives_mean_rest=(share_rest_geimpfte*mean_df_rest[BL_dict[BL]+"_Insgesamt"])*0.99
    series_of_lives_mean.extend([savedlives_mean_alter,savedlives_mean_rest])
    
    savedlives_min_alter=(share_alter_geimpfte*min_df_alter[BL_dict[BL]+"_Insgesamt"])*0.99
    savedlives_min_rest=(share_rest_geimpfte*min_df_rest[BL_dict[BL]+"_Insgesamt"])*0.99
    series_of_lives_min.extend([savedlives_min_alter,savedlives_min_rest])
    
    savedlives_max_alter=(share_alter_geimpfte*max_df_alter[BL_dict[BL]+"_Insgesamt"])*0.99
    savedlives_max_rest=(share_rest_geimpfte*max_df_rest[BL_dict[BL]+"_Insgesamt"])*0.99
    series_of_lives_max.extend([savedlives_max_alter,savedlives_max_rest])
    
    column_names.extend([BL_dict[BL]+"_alter",BL_dict[BL]+"_rest"])
    
saved_lives_mean_df=pd.concat(series_of_lives_mean,axis=1)   
saved_lives_mean_df.columns=column_names
saved_lives_max_df=pd.concat(series_of_lives_max,axis=1)   
saved_lives_max_df.columns=column_names
saved_lives_min_df=pd.concat(series_of_lives_min,axis=1)   
saved_lives_min_df.columns=column_names
saved_lives_min_df
print(saved_lives_min_df.sum(axis=1).iloc[-1],saved_lives_mean_df.sum(axis=1).iloc[-1],saved_lives_max_df.sum(axis=1).iloc[-1])
medrxiv_calc=saved_lives_mean_df
medrxiv_calc

<ipython-input-11-b540e01edb30>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  geimpfte_erst_indikation["indikation_rest_voll"]=geimpfte_erst_indikation["personen_erst_kumulativ"]-geimpfte_erst_indikation["indikation_alter_voll"]


91900.25436285831 115929.27772877351 145911.88586969732


,Berlin_alter,Berlin_rest,Brandenburg_alter,Brandenburg_rest,Baden-Württemberg_alter,Baden-Württemberg_rest,Bayern_alter,Bayern_rest,Bremen_alter,Bremen_rest,...,Schleswig-Holstein_alter,Schleswig-Holstein_rest,Saarland_alter,Saarland_rest,Sachsen_alter,Sachsen_rest,Sachsen-Anhalt_alter,Sachsen-Anhalt_rest,Thüringen_alter,Thüringen_rest
0,0.000000,13.877167,0.000000,1.100243,0.000000,18.289132,0.000000,30.066220,0.000000,3.770399,...,0.000000,8.197805,0.000000,2.133867,0.000000,2.063604,0.000000,17.457016,0.000000,1.117252
1,0.000000,32.764111,0.000000,1.177260,0.000000,41.431377,0.000000,46.301446,0.000000,5.314467,...,0.000000,25.282476,0.000000,8.555319,0.000000,9.717081,0.000000,46.254129,0.000000,2.900342
2,0.000000,77.527789,0.000000,33.403389,0.000000,111.007306,0.000000,250.235407,0.000000,15.180344,...,0.000000,56.553706,0.000000,21.031000,0.000000,30.081000,0.000000,117.231294,0.000000,9.141155
3,0.000000,94.852911,0.000000,35.416834,0.000000,149.946306,0.000000,336.725693,0.000000,15.629201,...,0.000000,73.668777,0.000000,26.956203,0.000000,37.303614,0.000000,129.372278,0.000000,9.141155
4,0.000000,112.118292,0.000000,35.416834,0.000000,175.914416,0.000000,346.040986,0.000000,15.629201,...,0.000000,80.701257,0.000000,32.911182,0.000000,45.353938,0.000000,136.626689,0.000000,9.141155
5,0.000000,124.740881,0.000000,35.416834,0.000000,211.176283,0.000000,513.077512,0.000000,16.491007,...,0.000000,90.520383,0.000000,41.178678,0.000000,49.243038,0.000000,148.825708,0.000000,9.141155
6,0.000000,151.556415,0.000000,36.407053,0.000000,240.935614,0.000000,587.821656,0.000000,17.083498,...,0.000000,96.843536,0.000000,41.178678,0.000000,55.173066,0.000000,155.139948,0.000000,9.141155
7,0.000000,165.476255,0.000000,36.682114,0.000000,282.428423,0.000000,690.893164,0.000000,23.816353,...,0.000000,131.509407,0.000000,47.997128,0.000000,87.759870,0.000000,181.395115,0.000000,30.639796
8,0.000000,187.375892,0.000000,59.457152,0.000000,332.082889,0.000000,734.124999,0.000000,28.681964,...,0.000000,172.427498,0.000000,55.907325,0.000000,122.081463,0.000000,198.747668,0.000000,41.880031
9,0.000000,206.186025,0.000000,90.021912,0.000000,376.480546,0.000000,748.319732,0.000000,34.337562,...,0.000000,203.455531,0.000000,64.621444,0.000000,147.468330,0.000000,204.562805,0.000000,60.297765


# C) [BMJ](https://gh.bmj.com/content/5/9/e003094#ref-7)
## And lastly we calculate all of this based numbers from a bmj paper.  Here we have to take into account the share of people with a comorbidity (and we already only look at IFR for high income countries)

In [12]:
#this is the number of people with 0, 1, or more than 2 comorbidities
conditions=pd.read_excel("RawData/conditions_share.xlsx",sheet_name="conditions_share",engine='openpyxl')
#we only take the absolute number of people in europe with a condition by leaving out the percentage
conditions.Europe=conditions.Europe.str.split()
conditions.Europe=conditions.Europe.str[0]
#we filter for the columns age, europe (pop number) and conditions
europe_with_conditions=conditions[["Age","Junges Alter","Altes Alter","Europe","Conditions"]]
#and we convert it into an integer
europe_with_conditions["Europe"]=europe_with_conditions["Europe"].astype(int)
europe_with_conditions

<ipython-input-12-0b9e71d53959>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  europe_with_conditions["Europe"]=europe_with_conditions["Europe"].astype(int)


,Age,Junges Alter,Altes Alter,Europe,Conditions
0,<15 years,0,15,117,0
1,15-49 years,15,49,274,0
2,50-54 years,50,54,33,0
3,55-59 years,55,59,29,0
4,60-64 years,60,64,23,0
5,65-69 years,65,69,16,0
6,70+ years,70,100,23,0
7,<15 years,0,15,3,1
8,15-49 years,15,49,49,1
9,50-54 years,50,54,14,1


In [13]:
#we filter for population with comorbidities and without and calculate the share of population that has comorbidities and that has not
comorbidty_pop=(europe_with_conditions[europe_with_conditions["Conditions"]>0].groupby(["Age","Junges Alter","Altes Alter"]).sum()/europe_with_conditions.groupby(["Age","Junges Alter","Altes Alter"]).sum()).reset_index()
comorbidty_pop=comorbidty_pop.rename(columns={"Europe":"Comorbity"})

no_comorbidty_pop=(europe_with_conditions[europe_with_conditions["Conditions"]==0].groupby(["Age","Junges Alter","Altes Alter"]).sum()/europe_with_conditions.groupby(["Age","Junges Alter","Altes Alter"]).sum()).reset_index()
no_comorbidty_pop=no_comorbidty_pop.rename(columns={"Europe":"No Comorbity"})

comorbities_europe=pd.concat([comorbidty_pop,no_comorbidty_pop["No Comorbity"]],axis=1).sort_values("Junges Alter")
comorbities_europe

,Age,Junges Alter,Altes Alter,Comorbity,Conditions,No Comorbity
6,<15 years,0,15,0.025000,1.0,0.975000
0,15-49 years,15,49,0.172205,1.0,0.827795
1,50-54 years,50,54,0.352941,1.0,0.647059
2,55-59 years,55,59,0.442308,1.0,0.557692
3,60-64 years,60,64,0.530612,1.0,0.469388
4,65-69 years,65,69,0.627907,1.0,0.372093
5,70+ years,70,100,0.770000,1.0,0.230000


## So now that we know which share of people in which age group has a comorbidity, we match this to our population data that says, how many people we have in each age group in each federal state.


In [14]:
#then again we get the population data split on gender
male_female_pop=pop_data[pop_data.columns[~pop_data.columns.str.contains("Insgesamt")]]
male_female_pop

,Alter_Alter,Baden-Württemberg_männlich,Baden-Württemberg_weiblich,Bayern_männlich,Bayern_weiblich,Berlin_männlich,Berlin_weiblich,Brandenburg_männlich,Brandenburg_weiblich,Bremen_männlich,...,Saarland_männlich,Saarland_weiblich,Sachsen_männlich,Sachsen_weiblich,Sachsen-Anhalt_männlich,Sachsen-Anhalt_weiblich,Schleswig-Holstein_männlich,Schleswig-Holstein_weiblich,Thüringen_männlich,Thüringen_weiblich
0,0,52735.45,50152.40,62372.25,59261.95,18932.55,17793.50,9589.30,9058.25,3507.40,...,3889.30,3701.20,16710.50,15946.70,8097.80,7670.30,12071.65,11457.00,7933.45,7823.25
1,1,53554.35,51118.55,62732.30,59821.50,18979.10,18092.75,10323.65,9690.95,3381.05,...,4009.95,3734.45,17565.50,16643.05,8498.70,8179.50,12516.25,11981.40,8574.70,8119.65
2,2,53667.40,50934.25,62644.90,60238.55,19172.90,18021.50,10779.65,10369.25,3347.80,...,4074.55,3912.10,18194.40,17459.10,8978.45,8483.50,12767.05,12077.35,8943.30,8494.90
3,3,53645.55,50778.45,62605.00,59439.60,18984.80,18266.60,11307.85,10735.00,3275.60,...,3990.95,3980.50,18676.05,17843.85,9178.90,8609.85,12895.30,12407.00,9163.70,8801.75
4,4,51484.30,49740.10,60378.20,57498.75,18245.70,17082.90,11090.30,10333.15,3260.40,...,4055.55,3633.75,18698.85,17462.90,8952.80,8618.40,12637.85,11964.30,9292.90,8673.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,86,13272.80,20798.40,14806.40,24276.80,3393.60,5784.80,3304.80,5664.00,738.40,...,1362.40,2228.00,5820.00,10155.20,2869.60,5529.60,3316.00,5472.00,2784.80,4820.00
87,87,11310.40,18943.20,12675.20,22424.00,2828.00,5004.80,2879.20,5151.20,591.20,...,1167.20,2117.60,4849.60,9032.80,2521.60,4902.40,2776.00,4832.80,2336.00,4482.40
88,88,9940.00,17228.80,10896.80,20076.00,2546.40,4580.80,2456.00,4632.00,540.80,...,1009.60,1935.20,4327.20,8572.00,2109.60,4519.20,2431.20,4621.60,1979.20,3972.00
89,89,8596.00,15994.40,9429.60,18464.00,2191.20,4408.80,2113.60,4300.00,452.80,...,848.00,1649.60,3838.40,8300.80,1789.60,4203.20,2103.20,4132.80,1649.60,3847.20


In [15]:
#again we build a list of series
with_comorbidity=[]
without_comorbidity=[]

#for each row in the population data
for index, row in male_female_pop.iterrows():
    #we retrieve the age of that row
    altersgruppe=row["Alter_Alter"]
    
    #and get the share of people with or without comorbidites in this age group
    comorbidity_shares=comorbities_europe[(comorbities_europe["Junges Alter"]<=altersgruppe)].iloc[-1]
    
    #and multiply it with the number of men and women in that age group
    with_comorbidity.append(row.iloc[1:]*comorbidity_shares["Comorbity"])
    without_comorbidity.append(row.iloc[1:]*comorbidity_shares["No Comorbity"])
    
#from these lists we build dataframes 
with_comorbidity = pd.concat(with_comorbidity, axis=1).T
without_comorbidity = pd.concat(without_comorbidity, axis=1).T
with_comorbidity=with_comorbidity.reset_index().rename(columns={"index":'Alter'})
without_comorbidity=without_comorbidity.reset_index().rename(columns={"index":'Alter'})
without_comorbidity  

,Alter,Baden-Württemberg_männlich,Baden-Württemberg_weiblich,Bayern_männlich,Bayern_weiblich,Berlin_männlich,Berlin_weiblich,Brandenburg_männlich,Brandenburg_weiblich,Bremen_männlich,...,Saarland_männlich,Saarland_weiblich,Sachsen_männlich,Sachsen_weiblich,Sachsen-Anhalt_männlich,Sachsen-Anhalt_weiblich,Schleswig-Holstein_männlich,Schleswig-Holstein_weiblich,Thüringen_männlich,Thüringen_weiblich
0,0,51417.06375,48898.59000,60812.94375,57780.40125,18459.23625,17348.66250,9349.56750,8831.79375,3419.71500,...,3792.06750,3608.67000,16292.73750,15548.03250,7895.35500,7478.54250,11769.85875,11170.57500,7735.11375,7627.66875
1,1,52215.49125,49840.58625,61163.99250,58325.96250,18504.62250,17640.43125,10065.55875,9448.67625,3296.52375,...,3909.70125,3641.08875,17126.36250,16226.97375,8286.23250,7975.01250,12203.34375,11681.86500,8360.33250,7916.65875
2,2,52325.71500,49660.89375,61078.77750,58732.58625,18693.57750,17570.96250,10510.15875,10110.01875,3264.10500,...,3972.68625,3814.29750,17739.54000,17022.62250,8753.98875,8271.41250,12447.87375,11775.41625,8719.71750,8282.52750
3,3,52304.41125,49508.98875,61039.87500,57953.61000,18510.18000,17809.93500,11025.15375,10466.62500,3193.71000,...,3891.17625,3880.98750,18209.14875,17397.75375,8949.42750,8394.60375,12572.91750,12096.82500,8934.60750,8581.70625
4,4,50197.19250,48496.59750,58868.74500,56061.28125,17789.55750,16655.82750,10813.04250,10074.82125,3178.89000,...,3954.16125,3542.90625,18231.37875,17026.32750,8728.98000,8402.94000,12321.90375,11665.19250,9060.57750,8456.66250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,86,3052.74400,4783.63200,3405.47200,5583.66400,780.52800,1330.50400,760.10400,1302.72000,169.83200,...,313.35200,512.44000,1338.60000,2335.69600,660.00800,1271.80800,762.68000,1258.56000,640.50400,1108.60000
87,87,2601.39200,4356.93600,2915.29600,5157.52000,650.44000,1151.10400,662.21600,1184.77600,135.97600,...,268.45600,487.04800,1115.40800,2077.54400,579.96800,1127.55200,638.48000,1111.54400,537.28000,1030.95200
88,88,2286.20000,3962.62400,2506.26400,4617.48000,585.67200,1053.58400,564.88000,1065.36000,124.38400,...,232.20800,445.09600,995.25600,1971.56000,485.20800,1039.41600,559.17600,1062.96800,455.21600,913.56000
89,89,1977.08000,3678.71200,2168.80800,4246.72000,503.97600,1014.02400,486.12800,989.00000,104.14400,...,195.04000,379.40800,882.83200,1909.18400,411.60800,966.73600,483.73600,950.54400,379.40800,884.85600


In [16]:
#and then create one bigger dataframe that includes
#->for all ages and all federal states
#->the estimated number of men and women with comorbitdies and without
pop_com=pd.merge(with_comorbidity,without_comorbidity,on="Alter",suffixes=["/with_com","/without_com"])
pop_com

,Alter,Baden-Württemberg_männlich/with_com,Baden-Württemberg_weiblich/with_com,Bayern_männlich/with_com,Bayern_weiblich/with_com,Berlin_männlich/with_com,Berlin_weiblich/with_com,Brandenburg_männlich/with_com,Brandenburg_weiblich/with_com,Bremen_männlich/with_com,...,Saarland_männlich/without_com,Saarland_weiblich/without_com,Sachsen_männlich/without_com,Sachsen_weiblich/without_com,Sachsen-Anhalt_männlich/without_com,Sachsen-Anhalt_weiblich/without_com,Schleswig-Holstein_männlich/without_com,Schleswig-Holstein_weiblich/without_com,Thüringen_männlich/without_com,Thüringen_weiblich/without_com
0,0,1318.38625,1253.81000,1559.30625,1481.54875,473.31375,444.83750,239.73250,226.45625,87.68500,...,3792.06750,3608.67000,16292.73750,15548.03250,7895.35500,7478.54250,11769.85875,11170.57500,7735.11375,7627.66875
1,1,1338.85875,1277.96375,1568.30750,1495.53750,474.47750,452.31875,258.09125,242.27375,84.52625,...,3909.70125,3641.08875,17126.36250,16226.97375,8286.23250,7975.01250,12203.34375,11681.86500,8360.33250,7916.65875
2,2,1341.68500,1273.35625,1566.12250,1505.96375,479.32250,450.53750,269.49125,259.23125,83.69500,...,3972.68625,3814.29750,17739.54000,17022.62250,8753.98875,8271.41250,12447.87375,11775.41625,8719.71750,8282.52750
3,3,1341.13875,1269.46125,1565.12500,1485.99000,474.62000,456.66500,282.69625,268.37500,81.89000,...,3891.17625,3880.98750,18209.14875,17397.75375,8949.42750,8394.60375,12572.91750,12096.82500,8934.60750,8581.70625
4,4,1287.10750,1243.50250,1509.45500,1437.46875,456.14250,427.07250,277.25750,258.32875,81.51000,...,3954.16125,3542.90625,18231.37875,17026.32750,8728.98000,8402.94000,12321.90375,11665.19250,9060.57750,8456.66250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,86,10220.05600,16014.76800,11400.92800,18693.13600,2613.07200,4454.29600,2544.69600,4361.28000,568.56800,...,313.35200,512.44000,1338.60000,2335.69600,660.00800,1271.80800,762.68000,1258.56000,640.50400,1108.60000
87,87,8709.00800,14586.26400,9759.90400,17266.48000,2177.56000,3853.69600,2216.98400,3966.42400,455.22400,...,268.45600,487.04800,1115.40800,2077.54400,579.96800,1127.55200,638.48000,1111.54400,537.28000,1030.95200
88,88,7653.80000,13266.17600,8390.53600,15458.52000,1960.72800,3527.21600,1891.12000,3566.64000,416.41600,...,232.20800,445.09600,995.25600,1971.56000,485.20800,1039.41600,559.17600,1062.96800,455.21600,913.56000
89,89,6618.92000,12315.68800,7260.79200,14217.28000,1687.22400,3394.77600,1627.47200,3311.00000,348.65600,...,195.04000,379.40800,882.83200,1909.18400,411.60800,966.73600,483.73600,950.54400,379.40800,884.85600


## Then we want to calculate the number of potential covid-deaths in each age group, based on these number of women and men per agegroup that have or have not comorbidities

In [17]:
#then we load in the ifr from the bmj stat
ifr_bmj=pd.read_excel("RawData/ifr.xlsx",sheet_name="bmj",engine='openpyxl')
ifr_bmj=ifr_bmj.rename(columns={
    "female":"weiblich/without_com",
    "male":"männlich/without_com",
    "female_com":"weiblich/with_com",
    "male_com":"männlich/with_com",
                                })
ifr_bmj

,Junges Alter,Altes Alter,weiblich/without_com,weiblich/with_com,männlich/without_com,männlich/with_com
0,0,19,0.00004,0.0361,0.00004,0.0397
1,20,29,0.00021,0.0534,0.00034,0.0963
2,30,39,0.00090,0.1364,0.00140,0.2100
3,40,49,0.00200,0.1847,0.00350,0.3057
4,50,59,0.01120,0.6353,0.01850,0.7865
5,60,69,0.04380,1.2395,0.11050,2.0008
6,70,79,0.17490,2.3906,0.47550,4.3483
7,80,100,1.17080,7.0568,3.77950,19.9581


In [18]:

#and now we calculate with this ifr table and the number of women and men with comorbidities dataframes that show
#how many women/men with comorbidities die in age each age group,
#how many women/men without comorbidities die in age each age group,
#if everyone gets covid

#again we build a list of series
mean_series_male_com=[]
mean_series_female_com=[]
mean_series_male_without_com=[]
mean_series_female_without_com=[]
ifr=ifr_bmj
for index, row in pop_com.iterrows():
    #we retrieve the age of that row
    altersgruppe=row["Alter"]
    #and then get the row from the nature paper that states the mortality in this age group
    mortality_numbers=ifr[(ifr["Junges Alter"]<=altersgruppe)].iloc[-1]
    
    #here we mulitply the ifr with the number of people fitting and then append the results to the list of series
    mean_series_male_com.append(((row[row.index.str.contains("männlich/with_com")]*mortality_numbers["männlich/with_com"]*0.01)))
    mean_series_female_com.append(((row[row.index.str.contains("weiblich/with_com")]*mortality_numbers["weiblich/with_com"]*0.01)))
    mean_series_male_without_com.append(((row[row.index.str.contains("männlich/without_com")]*mortality_numbers["männlich/without_com"]*0.01)))
    mean_series_female_without_com.append(((row[row.index.str.contains("weiblich/without_com")]*mortality_numbers["weiblich/without_com"]*0.01)))
#and we create four dataframes from these lists of series
mean_series_male_without_com = pd.concat(mean_series_male_without_com, axis=1).T
mean_series_female_without_com = pd.concat(mean_series_female_without_com, axis=1).T
mean_series_male_com = pd.concat(mean_series_male_com, axis=1).T
mean_series_female_com = pd.concat(mean_series_female_com, axis=1).T
mean_series_female_com


,Baden-Württemberg_weiblich/with_com,Bayern_weiblich/with_com,Berlin_weiblich/with_com,Brandenburg_weiblich/with_com,Bremen_weiblich/with_com,Hamburg_weiblich/with_com,Hessen_weiblich/with_com,Mecklenburg-Vorpommern_weiblich/with_com,Niedersachsen_weiblich/with_com,Nordrhein-Westfalen_weiblich/with_com,Rheinland-Pfalz_weiblich/with_com,Saarland_weiblich/with_com,Sachsen_weiblich/with_com,Sachsen-Anhalt_weiblich/with_com,Schleswig-Holstein_weiblich/with_com,Thüringen_weiblich/with_com
0,0.452625,0.534839,0.160586,0.081751,0.027985,0.085557,0.248853,0.052806,0.306863,0.706743,0.155802,0.033403,0.143919,0.069224,0.103399,0.070605
1,0.461345,0.539889,0.163287,0.087461,0.029228,0.084623,0.254923,0.055215,0.312753,0.728554,0.160955,0.033703,0.150204,0.073820,0.108132,0.073280
2,0.459682,0.543653,0.162644,0.093582,0.028199,0.083757,0.258413,0.056655,0.318978,0.728109,0.162190,0.035307,0.157568,0.076564,0.108998,0.076666
3,0.458276,0.536442,0.164856,0.096883,0.027899,0.085283,0.260582,0.058979,0.321216,0.739443,0.164076,0.035924,0.161041,0.077704,0.111973,0.079436
4,0.448904,0.518926,0.154173,0.093257,0.027067,0.078861,0.251014,0.059682,0.307455,0.708663,0.158734,0.032795,0.157603,0.077781,0.107978,0.078278
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,1130.130148,1319.137221,314.330760,307.766807,69.725700,161.577574,611.795204,213.393680,841.924791,1913.674879,440.610785,121.063638,551.806758,300.463866,297.334034,261.906075
87,1029.323478,1218.460961,271.947619,279.902609,64.118085,143.624510,555.979868,195.571026,772.546850,1750.097691,398.358054,115.064794,490.818505,266.383474,262.601593,243.561782
88,936.167508,1090.876839,248.908579,251.690652,57.554132,138.277714,515.074703,169.619503,721.556671,1620.122726,374.275736,105.153659,465.779850,245.561397,251.125543,215.827994
89,869.093471,1003.285015,239.562553,233.650648,55.771866,128.018820,495.817543,152.144608,681.042735,1534.356637,352.410382,89.634909,451.043558,228.390792,224.565441,209.046691


In [19]:
#then we create a dataframe for men and women that sums up the deaths with and without comorbidites
columnlist_female = [item[0] for item in mean_series_female_com.columns.str.split("/")]
columnlist_male = [item[0] for item in mean_series_male_com.columns.str.split("/")]

female_df=pd.DataFrame()
male_df=pd.DataFrame()
for i in columnlist_female:
    (i)
    female_df[(i)]=mean_series_female_com[i+"/with_com"]+mean_series_female_without_com[i+"/without_com"]
for i in columnlist_male:
    male_df[(i)]=mean_series_male_com[i+"/with_com"]+mean_series_male_without_com[i+"/without_com"]

#and again from this a dataframe that adds together the results for women and men
gesamt_df=pd.DataFrame()
columnlist_gesamt = [item[0] for item in male_df.columns.str.split("_")]
for i in columnlist_gesamt:
    gesamt_df[(i)+"_Insgesamt"]=male_df[i+"_männlich"]+female_df[i+"_weiblich"]

gesamt_df=gesamt_df.reset_index().rename(columns={"index":'Alter'})
gesamt_df

,Alter,Baden-Württemberg_Insgesamt,Bayern_Insgesamt,Berlin_Insgesamt,Brandenburg_Insgesamt,Bremen_Insgesamt,Hamburg_Insgesamt,Hessen_Insgesamt,Mecklenburg-Vorpommern_Insgesamt,Niedersachsen_Insgesamt,Nordrhein-Westfalen_Insgesamt,Rheinland-Pfalz_Insgesamt,Saarland_Insgesamt,Sachsen_Insgesamt,Sachsen-Anhalt_Insgesamt,Schleswig-Holstein_Insgesamt,Thüringen_Insgesamt
0,0,1.016151,1.201321,0.362815,0.184197,0.065373,0.191088,0.561894,0.118812,0.688144,1.590583,0.349073,0.074965,0.322507,0.155745,0.232387,0.155489
1,1,1.033694,1.210303,0.366113,0.197729,0.065366,0.191133,0.574116,0.123687,0.704363,1.638139,0.360554,0.076522,0.337882,0.164674,0.241910,0.164895
2,2,1.033125,1.213328,0.367441,0.208819,0.063950,0.189602,0.580020,0.126552,0.711797,1.644788,0.368447,0.078861,0.352053,0.172485,0.245400,0.172230
3,3,1.031433,1.205394,0.367808,0.217710,0.062892,0.189038,0.584164,0.130563,0.719580,1.658503,0.369591,0.078643,0.360643,0.175742,0.249827,0.177392
4,4,0.999364,1.164152,0.349040,0.211683,0.061868,0.176852,0.566500,0.133492,0.694220,1.598250,0.356434,0.076045,0.357292,0.173490,0.243003,0.177517
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,86,3341.244368,3788.629185,880.927880,859.620156,193.075323,453.278251,1799.229564,568.193868,2380.052678,5395.165496,1273.258416,348.276473,1524.147767,781.291777,850.488904,727.054153
87,87,2916.806621,3336.920217,744.607727,761.270304,163.288910,388.062538,1570.224359,511.298486,2096.934610,4793.915128,1116.811333,310.285672,1302.573009,689.017599,726.355281,634.929002
88,88,2596.521897,2914.254108,674.703468,662.945147,148.216203,354.003504,1396.486576,430.803431,1897.803857,4255.730017,991.385597,274.293817,1191.471290,600.267096,658.325251,547.887094
89,89,2307.898242,2584.091839,607.220372,588.415557,132.057033,318.296136,1285.217853,360.893737,1711.487073,3893.061476,894.915409,231.766964,1096.638137,530.287082,577.191457,487.252500


In [20]:
gesamt_df.sum()

Alter                                 4095.000000
Baden-Württemberg_Insgesamt          93434.593678
Bayern_Insgesamt                    109533.274750
Berlin_Insgesamt                     27926.716325
Brandenburg_Insgesamt                25192.546200
Bremen_Insgesamt                      5794.827740
Hamburg_Insgesamt                    13841.883410
Hessen_Insgesamt                     53146.761864
Mecklenburg-Vorpommern_Insgesamt     15985.843677
Niedersachsen_Insgesamt              71820.839786
Nordrhein-Westfalen_Insgesamt       155578.368549
Rheinland-Pfalz_Insgesamt            36593.726200
Saarland_Insgesamt                    9606.261742
Sachsen_Insgesamt                    42757.555572
Sachsen-Anhalt_Insgesamt             22948.146991
Schleswig-Holstein_Insgesamt         27239.734281
Thüringen_Insgesamt                  21706.364716
dtype: float64

In [31]:
#Now we want to calculate the number of saved lives
#we build a dictionary that connects the abreviations to the spelled out name of all federal staes
BL_dict={"BE":"Berlin","BB":"Brandenburg","BW":"Baden-Württemberg","BY":"Bayern",
         "HB":"Bremen","HE":"Hessen","HH":"Hamburg","MV":"Mecklenburg-Vorpommern",
         "NI":"Niedersachsen","NW":"Nordrhein-Westfalen","RP":"Rheinland-Pfalz","SH":"Schleswig-Holstein",
        "SL":"Saarland","SN":"Sachsen","ST":"Sachsen-Anhalt","TH":"Thüringen"}

#we split the population into two groups, people over 80 (because these are the ones 
#prioritized for vaccination by age 
pop_alter=pop_data[pop_data["Alter_Alter"]>=80].iloc[:,1:].sum()
#and the rest which is between 18 and 79 old and sum over these groups for each column
pop_rest=pop_data[(pop_data["Alter_Alter"]>=18)&(pop_data["Alter_Alter"]<80)].iloc[:,1:].sum()
print(pop_rest.iloc[-1])

#and we do the same for the potential covid deaths split by age group
mean_df_alter=gesamt_df[gesamt_df["Alter"]>=80].iloc[:,1:].sum()
mean_df_rest=gesamt_df[(gesamt_df["Alter"]>=18)&(mean_df["Alter"]<80)].iloc[:,1:].sum()

#these numbers we could actually also store as a factor, to update constantly a dashboard
coefficients_bmj=pd.concat([mean_df_alter,mean_df_rest], axis=1).reset_index().rename(columns={"index":"Bundesland",0:"Alter",1:"Rest"})
coefficients_bmj

1498951.5000000002


,Bundesland,Alter,Rest
0,Baden-Württemberg_Insgesamt,56593.517073,36806.463535
1,Bayern_Insgesamt,64995.513577,44498.314053
2,Berlin_Insgesamt,16340.219141,11576.234579
3,Brandenburg_Insgesamt,15270.381133,9915.049520
4,Bremen_Insgesamt,3491.324528,2301.521580
5,Hamburg_Insgesamt,8315.941119,5520.572282
6,Hessen_Insgesamt,31699.872197,21427.653957
7,Mecklenburg-Vorpommern_Insgesamt,9643.807291,6337.642456
8,Niedersachsen_Insgesamt,43213.692709,28582.197813
9,Nordrhein-Westfalen_Insgesamt,94213.523182,61309.287383


In [37]:

#again we build lists of series for mean, min, max
series_of_lives_mean=[]

#and also have one list for the column names
column_names=[]

#we go through the BL dictionary
for BL in (BL_dict.keys()):
    #retrieve the latest data for vaccinated people
    geimpfte=pd.read_csv("https://raw.githubusercontent.com/ard-data/2020-rki-impf-archive/master/data/9_csv_v2/region_"+BL+".csv")
    #and only filter for the sum of vaccinated people with two jabs 
    #and the subgroup of these people that were vaccinated because of age (over 80)
    geimpfte_erst_indikation=geimpfte[["personen_erst_kumulativ","indikation_alter_erst","personen_voll_kumulativ","indikation_alter_voll","impf_inzidenz_voll","impf_inzidenz_erst"]]
    #and we get the under 80 year olds by a simple substraction
    geimpfte_erst_indikation["indikation_rest_voll"]=geimpfte_erst_indikation["personen_voll_kumulativ"]-geimpfte_erst_indikation["indikation_alter_voll"]
    
    #and then we calculate which share of the over 80 population and which share of the other population has been vaccinated
    share_alter_geimpfte=geimpfte_erst_indikation["indikation_alter_voll"]/pop_alter[BL_dict[BL]+"_Insgesamt"]   
    share_rest_geimpfte=geimpfte_erst_indikation["indikation_rest_voll"]/pop_rest[BL_dict[BL]+"_Insgesamt"]
    
    #and calculate the savedlives by multiplying this share with the potential deaths in each age group for that federal state
    savedlives_mean_alter=(share_alter_geimpfte*mean_df_alter[BL_dict[BL]+"_Insgesamt"])*0.99
    savedlives_mean_rest=(share_rest_geimpfte*mean_df_rest[BL_dict[BL]+"_Insgesamt"])*0.99
    series_of_lives_mean.extend([savedlives_mean_alter,savedlives_mean_rest])
    
   
    column_names.extend([BL_dict[BL]+"_alter",BL_dict[BL]+"_rest"])
    
saved_lives_mean_df=pd.concat(series_of_lives_mean,axis=1)   
saved_lives_mean_df.columns=column_names
print("here",savedlives_mean_rest.iloc[-1])

print(saved_lives_min_df.sum(axis=1).iloc[-1],saved_lives_mean_df.sum(axis=1).iloc[-1])
bmj_calc=saved_lives_mean_df
bmj_calc

<ipython-input-37-f04ff6ed28ee>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  geimpfte_erst_indikation["indikation_rest_voll"]=geimpfte_erst_indikation["personen_voll_kumulativ"]-geimpfte_erst_indikation["indikation_alter_voll"]


here 139.80143627605523
91900.25436285831 59035.25130583212


,Berlin_alter,Berlin_rest,Brandenburg_alter,Brandenburg_rest,Baden-Württemberg_alter,Baden-Württemberg_rest,Bayern_alter,Bayern_rest,Bremen_alter,Bremen_rest,...,Schleswig-Holstein_alter,Schleswig-Holstein_rest,Saarland_alter,Saarland_rest,Sachsen_alter,Sachsen_rest,Sachsen-Anhalt_alter,Sachsen-Anhalt_rest,Thüringen_alter,Thüringen_rest
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [36]:
print(share_rest_geimpfte)


0     0.000066
1     0.000171
2     0.000540
3     0.000540
4     0.000540
5     0.000540
6     0.000540
7     0.001811
8     0.002476
9     0.003564
10    0.005333
11    0.007013
12    0.007198
13    0.008351
14    0.008713
15    0.009370
16    0.012829
17    0.014683
18    0.015620
19    0.017329
20    0.019389
21    0.021578
22    0.023295
23    0.024821
24    0.025987
25    0.027457
26    0.029196
27    0.030501
28    0.032000
29    0.033351
30    0.034899
31    0.035588
32    0.036573
33    0.037415
34    0.038618
35    0.039721
36    0.039831
37    0.040374
38    0.041558
39    0.042001
40    0.042632
41    0.043732
42    0.045068
43    0.046257
44    0.045635
45    0.046597
Name: indikation_rest_voll, dtype: float64


In [35]:
pop_rest

Baden-Württemberg_männlich           3948748.55
Baden-Württemberg_weiblich           3896716.85
Baden-Württemberg_Insgesamt          7845465.40
Bayern_männlich                      4693666.45
Bayern_weiblich                      4652075.00
Bayern_Insgesamt                     9345741.45
Berlin_männlich                      1309153.65
Berlin_weiblich                      1326472.15
Berlin_Insgesamt                     2635625.80
Brandenburg_männlich                  886344.70
Brandenburg_weiblich                  879223.25
Brandenburg_Insgesamt                1765567.95
Bremen_männlich                       242851.30
Bremen_weiblich                       240858.55
Bremen_Insgesamt                      483709.85
Hamburg_männlich                      652809.15
Hamburg_weiblich                      670118.45
Hamburg_Insgesamt                    1322927.60
Hessen_männlich                      2222353.25
Hessen_weiblich                      2230393.80
Hessen_Insgesamt                     445

In [23]:
#lastly we round down all these estimates to adjust for uncertainties 
#and making sure that this is understood as an estimate
bmj_calc=np.floor(bmj_calc/100)*100
nature_calc=np.floor(nature_calc/100)*100
medrxiv_calc=np.floor(medrxiv_calc/100)*100
bmj_calc

,Berlin_alter,Berlin_rest,Brandenburg_alter,Brandenburg_rest,Baden-Württemberg_alter,Baden-Württemberg_rest,Bayern_alter,Bayern_rest,Bremen_alter,Bremen_rest,...,Schleswig-Holstein_alter,Schleswig-Holstein_rest,Saarland_alter,Saarland_rest,Sachsen_alter,Sachsen_rest,Sachsen-Anhalt_alter,Sachsen-Anhalt_rest,Thüringen_alter,Thüringen_rest
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,100.0,0.0,200.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,100.0,0.0,300.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,100.0,0.0,300.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,100.0,0.0,300.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0
9,0.0,100.0,0.0,0.0,0.0,100.0,0.0,300.0,0.0,0.0,...,0.0,100.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0


In [24]:
bmj_calc

,Berlin_alter,Berlin_rest,Brandenburg_alter,Brandenburg_rest,Baden-Württemberg_alter,Baden-Württemberg_rest,Bayern_alter,Bayern_rest,Bremen_alter,Bremen_rest,...,Schleswig-Holstein_alter,Schleswig-Holstein_rest,Saarland_alter,Saarland_rest,Sachsen_alter,Sachsen_rest,Sachsen-Anhalt_alter,Sachsen-Anhalt_rest,Thüringen_alter,Thüringen_rest
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,100.0,0.0,200.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,100.0,0.0,300.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,100.0,0.0,300.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,100.0,0.0,300.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0
9,0.0,100.0,0.0,0.0,0.0,100.0,0.0,300.0,0.0,0.0,...,0.0,100.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0


In [25]:
nature_calc

,Berlin_alter,Berlin_rest,Brandenburg_alter,Brandenburg_rest,Baden-Württemberg_alter,Baden-Württemberg_rest,Bayern_alter,Bayern_rest,Bremen_alter,Bremen_rest,...,Schleswig-Holstein_alter,Schleswig-Holstein_rest,Saarland_alter,Saarland_rest,Sachsen_alter,Sachsen_rest,Sachsen-Anhalt_alter,Sachsen-Anhalt_rest,Thüringen_alter,Thüringen_rest
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,200.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,100.0,0.0,200.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,100.0,0.0,300.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,100.0,0.0,300.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,100.0,0.0,300.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
medrxiv_calc

,Berlin_alter,Berlin_rest,Brandenburg_alter,Brandenburg_rest,Baden-Württemberg_alter,Baden-Württemberg_rest,Bayern_alter,Bayern_rest,Bremen_alter,Bremen_rest,...,Schleswig-Holstein_alter,Schleswig-Holstein_rest,Saarland_alter,Saarland_rest,Sachsen_alter,Sachsen_rest,Sachsen-Anhalt_alter,Sachsen-Anhalt_rest,Thüringen_alter,Thüringen_rest
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,100.0,0.0,200.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,100.0,0.0,300.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0
4,0.0,100.0,0.0,0.0,0.0,100.0,0.0,300.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0
5,0.0,100.0,0.0,0.0,0.0,200.0,0.0,500.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0
6,0.0,100.0,0.0,0.0,0.0,200.0,0.0,500.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0
7,0.0,100.0,0.0,0.0,0.0,200.0,0.0,600.0,0.0,0.0,...,0.0,100.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0
8,0.0,100.0,0.0,0.0,0.0,300.0,0.0,700.0,0.0,0.0,...,0.0,100.0,0.0,0.0,0.0,100.0,0.0,100.0,0.0,0.0
9,0.0,200.0,0.0,0.0,0.0,300.0,0.0,700.0,0.0,0.0,...,0.0,200.0,0.0,0.0,0.0,100.0,0.0,200.0,0.0,0.0


In [27]:
#also we compare the coeefcients for mortality per age group (over 80 or not)
df=pd.merge(coefficients_bmj,coefficients_nature,on="Bundesland",suffixes=["_bmj","_nature"])
df=pd.merge(df,coefficients_medrxiv,on="Bundesland")
df=df.rename(columns={"Alter":"Alter_medrxiv","Rest":"Rest_medrxiv"})
df

,Bundesland,Alter_bmj,Rest_bmj,Alter_nature,Rest_nature,Alter_medrxiv,Rest_medrxiv
0,Baden-Württemberg_Insgesamt,56593.517073,36806.463535,44594.165504,32571.093485,97465.6032,69547.077891
1,Bayern_Insgesamt,64995.513577,44498.314053,51322.038504,39313.180753,112000.9400,83750.194949
2,Berlin_Insgesamt,16340.219141,11576.234579,12955.328520,10448.162991,27986.7864,22721.072515
3,Brandenburg_Insgesamt,15270.381133,9915.049520,12105.868048,8930.610315,25967.9760,19621.761867
4,Bremen_Insgesamt,3491.324528,2301.521580,2777.511536,2038.474497,6140.1024,4386.193164
5,Hamburg_Insgesamt,8315.941119,5520.572282,6598.362824,4939.848778,14537.5896,10582.036565
6,Hessen_Insgesamt,31699.872197,21427.653957,24994.721512,18918.090744,54775.6840,40304.666112
7,Mecklenburg-Vorpommern_Insgesamt,9643.807291,6337.642456,7694.834576,5646.836709,16734.7296,12360.386343
8,Niedersachsen_Insgesamt,43213.692709,28582.197813,34159.445920,25284.796150,74332.5680,54155.566319
9,Nordrhein-Westfalen_Insgesamt,94213.523182,61309.287383,74668.977480,54015.044454,165074.5240,115841.081806


In [28]:
#and we calculate how many people below 80 and over 80 have to get jabs to save one life
over_80_pop=pop_data[pop_data["Alter_Alter"]>=80].iloc[:,1:].sum()
under_80_pop=pop_data[(pop_data["Alter_Alter"]<80)&(pop_data["Alter_Alter"]>=18)].iloc[:,1:].sum()
under_80_pop

Baden-Württemberg_männlich           3948748.55
Baden-Württemberg_weiblich           3896716.85
Baden-Württemberg_Insgesamt          7845465.40
Bayern_männlich                      4693666.45
Bayern_weiblich                      4652075.00
Bayern_Insgesamt                     9345741.45
Berlin_männlich                      1309153.65
Berlin_weiblich                      1326472.15
Berlin_Insgesamt                     2635625.80
Brandenburg_männlich                  886344.70
Brandenburg_weiblich                  879223.25
Brandenburg_Insgesamt                1765567.95
Bremen_männlich                       242851.30
Bremen_weiblich                       240858.55
Bremen_Insgesamt                      483709.85
Hamburg_männlich                      652809.15
Hamburg_weiblich                      670118.45
Hamburg_Insgesamt                    1322927.60
Hessen_männlich                      2222353.25
Hessen_weiblich                      2230393.80
Hessen_Insgesamt                     445

In [29]:
Bundesland="Berlin"
impfungen_life_ratio_alt=1/((df[df["Bundesland"]==Bundesland+str("_Insgesamt")]["Alter_bmj"])/over_80_pop[str(Bundesland)+"_Insgesamt"])
impfungen_life_ratio_jung=1/((df[df["Bundesland"]==Bundesland+str("_Insgesamt")]["Rest_bmj"])/under_80_pop[str(Bundesland)+"_Insgesamt"])
print(impfungen_life_ratio_alt,impfungen_life_ratio_jung)

2    10.341452
Name: Alter_bmj, dtype: float64 2    227.675569
Name: Rest_bmj, dtype: float64


In [30]:
Bundesland="Brandenburg"
impfungen_life_ratio_alt=1/((df[df["Bundesland"]==Bundesland+str("_Insgesamt")]["Alter_bmj"])/over_80_pop[str(Bundesland)+"_Insgesamt"])
impfungen_life_ratio_jung=1/((df[df["Bundesland"]==Bundesland+str("_Insgesamt")]["Rest_bmj"])/under_80_pop[str(Bundesland)+"_Insgesamt"])
print(impfungen_life_ratio_alt,impfungen_life_ratio_jung)

3    10.33876
Name: Alter_bmj, dtype: float64 3    178.069504
Name: Rest_bmj, dtype: float64
